In [1]:
from block_methods import *
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker, cm
import matplotlib.colors as colors
from scipy import integrate

%load_ext autoreload
%autoreload 2

In [2]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis([0, 0.6, 0.8]))

In [3]:
n = 1000 # matrix dimension
b = 4 # block size
step = 5
thresh = 1e-8

# set up matrix
Λ = np.linspace(1e-2,1,n)
H = np.diag(Λ)

λmin = np.min(Λ)
λmax = np.max(Λ)

f = lambda x: np.sqrt(x)

K = 30 # max iter
w = 0

angles = (1-np.linspace(0.01,.99,30))*np.pi # angle for D contour
r = np.linspace(1.01,1.15,25) # radius for D contour
lim = 10
np.random.seed(0)

In [5]:
error_FAr = []
error_wLSr = []
error_absr = []
error_fullr = []

V = np.random.randn(n,b)

# run Lanczos
Q,Qkp1,A,B,B_0 = block_lanczos(H, V, K, K+1)

# generate tridiagonal matrix
T = get_block_tridiag(A,B)

orthTest(Q, b, K, thresh)
threeTermTest(H, Q, T, Qkp1, B, b, K, thresh)

fAV = np.diag(f(Λ))@V

T = get_block_tridiag(A[:K],B[:K])
Eval,Evec = np.linalg.eigh(T)

error_wLS = h_norm(np.diag(1/(Λ-w))@V - get_lan_wLS(Eval, Evec, b, B_0, w, Q, K), Λ, h_w, w)

lanf = Q[:,:b*K]@(Evec@np.diag(f(Eval))@Evec.T@Ei(b*K,b,1)@B_0)
error_FA = h_norm(fAV - lanf, Λ, h_w, w)
    
error_abs = np.zeros((len(r),len(angles)))
error_full = np.zeros((len(r),len(angles)))

for i in range(len(r)):
    for l in range(len(angles)):
        
        pts = np.logspace(-15, -1, lim)
        error_abs[i,l] = get_trig_ineq_bound(pts, angles[l], r[i], Eval, Evec, b, B_0, λmin, f, λmin/100, w, Λ, V, Q, K)
        error_full[i,l] = get_a_posteriori_bound(pts, angles[l], r[i], Eval, Evec, b, B_0, λmin, f, λmin/100, w, λmax)

/src/notebooks/block_lanczos_CIF/block_methods/block_methods.py:289: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  result += sp.integrate.quad(trig_ineq_bound_integrand, 0, 1, args=(Γl, angle, r, Eval, Evec, b, B_0, λmin, f, c, w, Λ, V, Q, k), points = pts)[0]
/src/notebooks/block_lanczos_CIF/block_methods/block_methods.py:296: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be

KeyboardInterrupt: 

In [ ]:
np.save("data/figure_3.5/error_abs", error_abs)
np.save("data/figure_3.5/error_full", error_full)

# Uncomment the cell below to generate figures directly from data. 

In [ ]:
# np.save("data/figure_3.5/error_abs", error_abs)
# np.save("data/figure_3.5/error_full", error_full)

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(9,3),subplot_kw={'projection': 'polar'})

im = axes[0].pcolormesh(np.pi-angles,(r-1)*20+1,error_abs/error_FA,norm=colors.LogNorm(vmin=1e0,vmax=1e7),rasterized=True)
axes[1].pcolormesh(np.pi-angles,(r-1)*20+1,error_full/error_FA,norm=colors.LogNorm(vmin=1e0,vmax=1e7),rasterized=True)
axes[0].text(.5,.1,f'triangle inequality',ha='center', va='top', transform=axes[0].transAxes,bbox=dict(facecolor='white',edgecolor='none',pad=2))
axes[1].text(.5,.1,f'computable bound',ha='center', va='top', transform=axes[1].transAxes,bbox=dict(facecolor='white',edgecolor='none',pad=2))


for i in range(2):
    axes[i].set_yticks((np.array([1,1.05,1.1])-1)*20+1)
    axes[i].set_yticklabels([1,1.05,1.1])

    axes[i].set_thetamin(0)
    axes[i].set_thetamax(180)
    
    axes[i].set_xticks([0,np.pi/4,np.pi/2,3*np.pi/4,np.pi])
    axes[i].set_xticklabels([0,'$\pi/4$','$\pi/2$','$3\pi/4$','$\pi$'])

plt.colorbar(im,ax=axes.ravel().tolist(),shrink=.9,pad=0.08)

plt.savefig("imgs/figure_3.5.pdf")